[View in Colaboratory](https://colab.research.google.com/github/shashank2806/Apna-Broker-API/blob/master/Banglore_house_rent_prediction.ipynb)

In [0]:
from google.colab import files
files.upload()

In [3]:
!unzip "hackathon_rentomter_nobroker.csv.zip"

Archive:  hackathon_rentomter_nobroker.csv.zip
  inflating: hackathon_rentomter_nobroker.csv  


In [0]:
import pandas as pd
import pickle

In [0]:
data = pd.read_csv('hackathon_rentomter_nobroker.csv')

In [0]:
features_corelated = ['longitude', 'latitude', 'gym', 'lift', 'swimming_pool', 'property_size', 'bathroom', 'floor', 'total_floor', 'balconies', 'building_type', 'parking', 'type']

In [0]:
y = data['rent']

In [0]:
one_hot_encoded_features = pd.get_dummies(data[features_corelated])

In [10]:
one_hot_encoded_features.describe()

,longitude,latitude,gym,lift,swimming_pool,property_size,bathroom,floor,total_floor,balconies,...,parking_BOTH,parking_FOUR_WHEELER,parking_NONE,parking_TWO_WHEELER,type_BHK1,type_BHK2,type_BHK3,type_BHK4,type_BHK4PLUS,type_RK1
count,25000.000000,25000.000000,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.00000,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,77.637447,12.945900,0.22608,0.362440,0.174600,1063.764160,1.861120,1.881600,3.810480,1.15212,...,0.528680,0.069400,0.051160,0.350760,0.176360,0.568120,0.215720,0.011960,0.001520,0.026320
std,0.111732,0.029613,0.41830,0.480715,0.379632,591.174731,0.759589,2.187879,3.201751,1.00891,...,0.499187,0.254138,0.220328,0.477217,0.381134,0.495348,0.411329,0.108708,0.038958,0.160088
min,77.500072,12.900004,0.00000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,77.571956,12.918511,0.00000,0.000000,0.000000,700.000000,1.000000,1.000000,2.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,77.633363,12.943927,0.00000,0.000000,0.000000,1050.000000,2.000000,1.000000,3.000000,1.00000,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,77.695972,12.971114,0.00000,1.000000,0.000000,1290.000000,2.000000,3.000000,4.000000,2.00000,...,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,80.266346,12.999999,1.00000,1.000000,1.000000,50000.000000,34.000000,25.000000,50.000000,22.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
# Split into validation and training data
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(one_hot_encoded_features, y, test_size=0.2)



In [9]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
xgb_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(val_X, val_y)], verbose=False)
val_predictions = xgb_model.predict(val_X)



from sklearn.metrics import mean_absolute_error
train_accuracy = xgb_model.score(train_X, train_y)
val_accuracy = xgb_model.score(val_X, val_y)
print('train_accuracy: ',train_accuracy)
print('val_accuracy: ',val_accuracy)

train_accuracy:  0.8089212641671288
val_accuracy:  0.7968190449224496


In [0]:
# save model
# save the classifier
with open('model.pkl', 'wb') as fid:
    pickle.dump(xgb_model, fid)    

# load it again
with open('model.pkl', 'rb') as fid:
    xgb_model_saved = pickle.load(fid)

In [11]:
!ls

hackathon_rentomter_nobroker.csv      model.pkl
hackathon_rentomter_nobroker.csv.zip  sample_data


In [0]:
files.download('model.pkl')

In [13]:
data[features_corelated].head()

,longitude,latitude,gym,lift,swimming_pool,property_size,bathroom,floor,total_floor,balconies,building_type,parking,type
0,77.576914,12.936601,0,0,0,1000,2,2,2.0,1.0,IF,TWO_WHEELER,BHK2
1,77.545219,12.997989,0,0,0,1218,3,0,1.0,0.0,IH,BOTH,BHK2
2,77.587642,12.935696,0,1,0,1820,3,4,9.0,2.0,AP,BOTH,BHK3
3,77.656118,12.953507,0,1,0,1100,2,4,4.0,1.0,AP,BOTH,BHK2
4,77.742436,12.968520,1,1,1,1475,2,1,9.0,2.0,AP,BOTH,BHK2


In [14]:
data.iloc[0]

id                                  ff8081815917971401591af8895032d0
type                                                            BHK2
locality                                                  Jayanagara
activation_date                                  2018-07-05 17:09:49
latitude                                                     12.9366
longitude                                                    77.5769
lease_type                                                    FAMILY
gym                                                                0
lift                                                               0
swimming_pool                                                      0
negotiable                                                         0
furnishing                                            SEMI_FURNISHED
parking                                                  TWO_WHEELER
property_size                                                   1000
property_age                      

In [15]:
one_hot_encoded_features.columns

Index(['longitude', 'latitude', 'gym', 'lift', 'swimming_pool',
       'property_size', 'bathroom', 'floor', 'total_floor', 'balconies',
       'building_type_AP', 'building_type_GC', 'building_type_IF',
       'building_type_IH', 'parking_BOTH', 'parking_FOUR_WHEELER',
       'parking_NONE', 'parking_TWO_WHEELER', 'type_BHK1', 'type_BHK2',
       'type_BHK3', 'type_BHK4', 'type_BHK4PLUS', 'type_RK1'],
      dtype='object')

In [16]:
one_hot_encoded_features.describe(include='all')

,longitude,latitude,gym,lift,swimming_pool,property_size,bathroom,floor,total_floor,balconies,...,parking_BOTH,parking_FOUR_WHEELER,parking_NONE,parking_TWO_WHEELER,type_BHK1,type_BHK2,type_BHK3,type_BHK4,type_BHK4PLUS,type_RK1
count,25000.000000,25000.000000,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.00000,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,77.637447,12.945900,0.22608,0.362440,0.174600,1063.764160,1.861120,1.881600,3.810480,1.15212,...,0.528680,0.069400,0.051160,0.350760,0.176360,0.568120,0.215720,0.011960,0.001520,0.026320
std,0.111732,0.029613,0.41830,0.480715,0.379632,591.174731,0.759589,2.187879,3.201751,1.00891,...,0.499187,0.254138,0.220328,0.477217,0.381134,0.495348,0.411329,0.108708,0.038958,0.160088
min,77.500072,12.900004,0.00000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,77.571956,12.918511,0.00000,0.000000,0.000000,700.000000,1.000000,1.000000,2.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,77.633363,12.943927,0.00000,0.000000,0.000000,1050.000000,2.000000,1.000000,3.000000,1.00000,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,77.695972,12.971114,0.00000,1.000000,0.000000,1290.000000,2.000000,3.000000,4.000000,2.00000,...,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,80.266346,12.999999,1.00000,1.000000,1.000000,50000.000000,34.000000,25.000000,50.000000,22.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
val_X.iloc[0], val_y.iloc[0]

(longitude                 77.593236
 latitude                  12.913533
 gym                        0.000000
 lift                       1.000000
 swimming_pool              0.000000
 property_size           1720.000000
 bathroom                   2.000000
 floor                      1.000000
 total_floor                3.000000
 balconies                  3.000000
 building_type_AP           1.000000
 building_type_GC           0.000000
 building_type_IF           0.000000
 building_type_IH           0.000000
 parking_BOTH               1.000000
 parking_FOUR_WHEELER       0.000000
 parking_NONE               0.000000
 parking_TWO_WHEELER        0.000000
 type_BHK1                  0.000000
 type_BHK2                  0.000000
 type_BHK3                  1.000000
 type_BHK4                  0.000000
 type_BHK4PLUS              0.000000
 type_RK1                   0.000000
 Name: 6439, dtype: float64, 33000)

In [18]:
xgb_model.predict(val_X.iloc[0:1])

array([29105.078], dtype=float32)

In [19]:
val_X.iloc[0:1].shape

(1, 24)

In [0]:
import numpy as np

In [21]:
qw = np.array([1,2])
qw.shape

(2,)

In [0]:
c = pd.DataFrame([77,12,0,0,0,500,1,1,2,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0])

In [0]:
c=c.transpose()

In [24]:
c.shape

(1, 24)

In [0]:
c.columns = one_hot_encoded_features.columns

In [88]:
xgb_model.predict(c)

array([7233.848], dtype=float32)